In [274]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
import statistics

In [275]:
gnbc=GaussianNB()
knnc=KNeighborsClassifier(n_neighbors=3)
lrc=LogisticRegression(solver='liblinear')
f=pd.read_csv('diabetes.csv')
splitted_data=kf.split(f)

In [276]:
nb=[]
knn=[]
lr=[]
for train,test in splitted_data:
    gnbc.fit(f.iloc[train[:],0:8],f.iloc[train[:],8])
    knnc.fit(f.iloc[train[:],0:8],f.iloc[train[:],8])
    lrc.fit(f.iloc[train[:],0:8],f.iloc[train[:],8])
    tem=gnbc.predict(f.iloc[test[:],0:8])
    actual=list(f.iloc[test[:],8])
    err=actual-tem
    nb.append(sum(err)/len(test))
    tem=knnc.predict(f.iloc[test[:],0:8])
    actual=f.iloc[test[:],8]
    err=actual-tem
    knn.append(sum(err)/len(test))
    tem=lrc.predict(f.iloc[test[:],0:8])
    actual=f.iloc[test[:],8]
    err=actual-tem
    lr.append(sum(err)/len(test))

for i in range(0,10):
    print("\nFOLD ",i+1)
    print("Naive Bayes: ",nb[i])
    print("Logistic: ",lr[i])
    print("Knn: ",knn[i])


FOLD  1
Naive Bayes:  0.05194805194805195
Logistic:  0.12987012987012986
Knn:  0.0

FOLD  2
Naive Bayes:  0.0
Logistic:  0.05194805194805195
Knn:  0.012987012987012988

FOLD  3
Naive Bayes:  -0.05194805194805195
Logistic:  0.05194805194805195
Knn:  -0.07792207792207792

FOLD  4
Naive Bayes:  -0.03896103896103896
Logistic:  0.0
Knn:  -0.1038961038961039

FOLD  5
Naive Bayes:  -0.09090909090909091
Logistic:  0.06493506493506493
Knn:  -0.03896103896103896

FOLD  6
Naive Bayes:  -0.05194805194805195
Logistic:  -0.012987012987012988
Knn:  -0.03896103896103896

FOLD  7
Naive Bayes:  0.2077922077922078
Logistic:  0.23376623376623376
Knn:  0.12987012987012986

FOLD  8
Naive Bayes:  0.09090909090909091
Logistic:  0.14285714285714285
Knn:  0.09090909090909091

FOLD  9
Naive Bayes:  0.10526315789473684
Logistic:  0.14473684210526316
Knn:  0.14473684210526316

FOLD  10
Naive Bayes:  0.013157894736842105
Logistic:  0.039473684210526314
Knn:  0.0


In [277]:
#################   ANSWER 2  #################

import scipy.stats as stats
import numpy as np
nb=np.abs(nb)
mean=np.mean(nb)
std_dev=np.std(nb)
t=np.sqrt(10)*(mean-0.2)/std_dev
df=10-1
t=stats.t.ppf(0.9,df)
print("T critical:",t)
print("T value:",t)
if t>t:
    print("No")
else:
    print("Yes")

T critical: 1.3830287383964925
T value: 1.3830287383964925
Yes


In [278]:
################# ANSWER 3 #################

t=np.sqrt(10)*mean/std_dev
tc=stats.t.ppf(0.95/2,df)
diff=nb-knn
mean=np.mean(diff)
std_dev=np.std(diff)
print("T value:",t)
print("T critical:",tc)
if t>=tc and t<=-tc:
    print("error is same")
else:
    print("different errors values")

T value: 3.980571757324566
T critical: -0.06447679010158362
different errors values


In [279]:
################# ANSWER 4 #################
et=np.array(np.transpose([knn,nb,lr]))
nm=np.shape(et)[1]
classifier_error_average=np.mean(et,axis=0)
ew=np.sum(np.var(et,axis=0)/nm)
eb=10*np.var(classifier_error_average)
f=stats.f.ppf(0.95,dfn=nm-1,dfd=nm*(10-1))
if eb/ew<f:
    print("same errors")
else:
    print("different errors")
print("value of Estimators ratio:",eb/ew)
print("value of f:",f)

same errors
value of Estimators ratio: 2.0015640596710798
value of f: 3.3541308285291986


In [262]:
###############  ANSWER 5  ###############
class Naive:
    def __init__(self):
        self.mean=[]
        self.variance=[]
        self.prior=[]
    def con_probability(self,x):
        result=[]
        for i in range(self.mean.shape[0]):
            num=np.exp(-((x-self.mean[i])**2)/(2*self.variance[i]))
            den=np.sqrt(2*np.pi*self.variance[i])
            result.append((num/den).item())
        return np.array(result)
    def fit(self,x_train,y_train):
        no_of_classes=np.unique(y_train).shape[0]
        for i in range(no_of_classes):
            rd=x_train[y_train==i]
            self.prior.append(rd.shape[0]/x_train.shape[0])
            self.mean.append(np.mean(rd))
            self.variance.append(np.var(rd))
        self.mean=np.array(self.mean)
        self.variance=np.array(self.variance)
        self.prior=np.array(self.prior)
    def predict(self,x):
        pre=[]
        for x_data in x:
            prob=self.con_probability(x_data)*self.prior
            pre.append(np.argmax(prob))
        return np.array(pre)

In [263]:
frame=pd.read_csv('diabetes.csv')
splitted_data=kf.split(frame)
x=frame.loc[:,["Glucose"]].values
y=frame.iloc[:,-1].values
skerr=[]
cnerr=[]
kf=KFold(n_splits=10, random_state=None, shuffle=True)
for train,validation in kf.split(frame):
    cm=Naive()
    sm=GaussianNB()
    cm.fit(x[train],y[train])
    pre=cm.predict(x[validation])
    error=np.sum(pre!=y[validation])/np.shape(y[validation])[0]
    cnerr.append(error)
    sm.fit(x[train],y[train])
    pre=sm.predict(x[validation])
    error=np.sum(pre!=y[validation])/np.shape(y[validation])[0]
    skerr.append(error)
res=pd.DataFrame({"Library model":skerr,"Custom model":cnerr})
print(res)

   Library model  Custom model
0       0.311688      0.311688
1       0.220779      0.220779
2       0.246753      0.246753
3       0.272727      0.272727
4       0.246753      0.246753
5       0.246753      0.246753
6       0.233766      0.233766
7       0.259740      0.259740
8       0.197368      0.197368
9       0.263158      0.263158
